In [1]:
#importando as bibliotecas
import folium
import json
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
import xml.etree.cElementTree as etree

/home/vinicius.vancellote/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Abrindo o Arquivo GeoJson com o limite das Zonas Pluviometricas
zonasgpd = gpd.read_file('/home/vinicius.vancellote/Codigos_Python/Bases/EstacoesPluviometricas/Zonas_Pluviometricas.geojson')

In [3]:
zonasgpd.columns

Index(['OBJECTID', 'Name', 'Endereço', 'Est', 'Cod', 'ShapeSTArea',
       'ShapeSTLength', 'geometry'],
      dtype='object')

In [4]:
zonasgpd = zonasgpd.drop(columns=['Endereço','Est','ShapeSTArea','ShapeSTLength','Cod'])

In [5]:
#bus = '/home/vinicius.vancellote/data/json/Agosto.json'
#file = open(bus, encoding="utf8")

In [6]:
#with open(bus, encoding="utf8") as file: 
#    data = json.load(file)

In [7]:
#Transformando em Dataframe
#dfbus = pd.json_normalize(data)

In [8]:
dfbus = pd.read_csv("/home/vinicius.vancellote/data/json/Agosto.csv")

In [9]:
dfbus = dfbus.drop(columns = ['Unnamed: 0'])
dfbus.head()

,DATE,BUSID,LINE,LATITUDE,LONGITUDE,VELOCITY
0,08-11-2019 16:20:00,C13161,878.0,-22.880409,-43.358429,0.0
1,08-11-2019 14:19:15,C27058,SE940,-22.842400,-43.324741,0.0
2,08-11-2019 04:50:53,D13384,794.0,-22.879129,-43.468849,6.0
3,08-11-2019 11:41:12,D86073,852.0,-22.901430,-43.554539,0.0
4,08-11-2019 22:35:43,C51589,371.0,-22.904261,-43.356949,0.0


In [10]:
dfbus['LATITUDE'] = dfbus['LATITUDE'].astype(str)
dfbus['LONGITUDE'] = dfbus['LONGITUDE'].astype(str)

In [11]:
#Criando uma coluna chamada Geometry para ficar similar ao GeoJson
dfbus = dfbus.assign(geometry = ('POINT Z (' + dfbus['LONGITUDE'] + ' ' + dfbus['LATITUDE'] + ' ' + '0.00000)'))

In [12]:
#Transformando o dfbus em um GeoJson com a mesma codificação do ZonasPluviometricas
cp_union = gpd.GeoDataFrame(
    dfbus.loc[:, [c for c in dfbus.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(dfbus["geometry"]),
    crs="epsg:4326",
)

In [13]:
#Fazendo o Join com a biblioteca GeoJson SJOIN
dfjoin = gpd.sjoin(cp_union,zonasgpd,how='left')

In [14]:
dfjoin

,DATE,BUSID,LINE,LATITUDE,LONGITUDE,VELOCITY,geometry,index_right,OBJECTID,Name
0,08-11-2019 16:20:00,C13161,878.0,-22.880409,-43.358429,0.00,POINT Z (-43.35843 -22.88041 0.00000),9.0,10.0,Região 10
1,08-11-2019 14:19:15,C27058,SE940,-22.8424,-43.324741,0.00,POINT Z (-43.32474 -22.84240 0.00000),10.0,11.0,Região 11
2,08-11-2019 04:50:53,D13384,794.0,-22.879129,-43.468849,6.00,POINT Z (-43.46885 -22.87913 0.00000),11.0,12.0,Região 12
3,08-11-2019 11:41:12,D86073,852.0,-22.90143,-43.554539,0.00,POINT Z (-43.55454 -22.90143 0.00000),25.0,26.0,Região 26
4,08-11-2019 22:35:43,C51589,371.0,-22.904261,-43.356949,0.00,POINT Z (-43.35695 -22.90426 0.00000),13.0,14.0,Região 14
...,...,...,...,...,...,...,...,...,...,...
40340349,08-27-2019 01:11:20,B31084,485.0,-22.8048,-43.309959,0.00,POINT Z (-43.30996 -22.80480 0.00000),10.0,11.0,Região 11
40340350,08-27-2019 09:49:46,D86074,802.0,-22.88615,-43.474171,36.00,POINT Z (-43.47417 -22.88615 0.00000),11.0,12.0,Região 12
40340351,08-27-2019 20:07:06,B63090,919.0,-22.804399,-43.363571,0.00,POINT Z (-43.36357 -22.80440 0.00000),10.0,11.0,Região 11
40340352,08-27-2019 21:57:45,C30233,731.0,-22.883221,-43.55302,0.00,POINT Z (-43.55302 -22.88322 0.00000),25.0,26.0,Região 26


In [15]:
chuva = pd.read_csv("/home/vinicius.vancellote/data/json/chuva.csv")

In [16]:
chuva.tail()

,Unnamed: 0,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao
196356,196356,33_2019-12-24 03:45:00,33,0.0,0.0,0.8,29.2,44.0,03:45:00,2019-12-24
196357,196357,33_2019-12-24 04:30:00,33,0.0,0.0,0.8,29.2,44.0,04:30:00,2019-12-24
196358,196358,33_2019-12-24 04:00:00,33,0.0,0.0,0.8,29.2,44.0,04:00:00,2019-12-24
196359,196359,33_2019-12-24 03:30:00,33,0.0,0.0,0.8,29.2,44.0,03:30:00,2019-12-24
196360,196360,33_2019-12-24 04:15:00,33,0.0,0.0,0.8,29.2,44.0,04:15:00,2019-12-24


In [17]:
chuva = chuva.drop(columns=['Unnamed: 0'])

In [18]:
import datetime as dt

In [19]:
chuva['data_particao'] = pd.to_datetime(chuva['data_particao'],format = '%Y-%m-%d' )

In [20]:
chuva['horario'] = pd.to_datetime(chuva['horario'],format = '%H:%M:%S').dt.time

In [21]:
chuva['DATE'] = chuva.apply(lambda row: pd.datetime.combine(row['data_particao'],row['horario']),axis=1)

<ipython-input-21-958b6bf7996f>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  chuva['DATE'] = chuva.apply(lambda row: pd.datetime.combine(row['data_particao'],row['horario']),axis=1)


In [22]:
chuva['id_estacao']=chuva['id_estacao'].astype(float)
chuva['id_estacao']=chuva['id_estacao'].astype(str)
dfjoin['OBJECTID']=dfjoin['OBJECTID'].astype(str)

In [23]:
chuva = chuva.sort_values(by=['DATE','id_estacao'])
dfjoin = dfjoin.sort_values(by=['DATE','OBJECTID'])

In [25]:
dfjoin['DATE'] = pd.to_datetime(dfjoin['DATE'],format = '%m-%d-%Y %H:%M:%S')

In [26]:
final_df = pd.merge_asof(dfjoin,chuva,on=['DATE'],left_by = ['OBJECTID'],right_by = ['id_estacao'],direction = 'nearest')

In [27]:
final_df.head()

,DATE,BUSID,LINE,LATITUDE,LONGITUDE,VELOCITY,geometry,index_right,OBJECTID,Name,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao
0,2019-08-03 00:00:01,D86378,884.0,-22.98399,-43.693539,26.0,POINT Z (-43.69354 -22.98399 0.00000),26.0,27.0,Região 27,27_2019-08-03 00:00:00,27.0,0.0,0.0,0.0,0.0,0.0,00:00:00,2019-08-03
1,2019-08-03 00:00:01,A48002,539.0,-22.910109,-43.270721,0.0,POINT Z (-43.27072 -22.91011 0.00000),6.0,7.0,Região 7,7_2019-08-03 00:00:00,7.0,0.0,0.0,0.0,0.0,0.0,00:00:00,2019-08-03
2,2019-08-03 00:00:01,C41889,2012.0,-22.84944,-43.76722,0.0,POINT Z (-43.76722 -22.84944 0.00000),NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,2019-08-03 00:00:02,D13224,803.0,-22.879551,-43.45673,0.0,POINT Z (-43.45673 -22.87955 0.00000),11.0,12.0,Região 12,12_2019-08-03 00:00:00,12.0,0.0,0.0,0.0,0.0,0.0,00:00:00,2019-08-03
4,2019-08-03 00:00:03,B42512,NaN,-22.876369,-43.33036,0.0,POINT Z (-43.33036 -22.87637 0.00000),9.0,10.0,Região 10,10_2019-08-03 00:00:00,10.0,0.0,0.0,0.0,0.0,0.0,00:00:00,2019-08-03


In [28]:
final_df.to_csv("/home/vinicius.vancellote/data/json/DF_FINAL_AGO.csv", sep = ',')